# Introduction
This file will be used to format the training set for the different tests

We will begin by clearing folders that have been created by this same script. So we can get a fresh setup

In [ ]:
import os
# !pip install -U ultralytics
import ultralytics
from ultralytics import settings
# Get the current working directory
datasets_folder = os.path.join(os.getcwd(), "datasets")
# Update the datasets_dir setting with the actual path value
if not os.path.exists(datasets_folder):
    os.makedirs(datasets_folder)
settings.update({"datasets_dir": datasets_folder})
!yolo checks

In [ ]:
%reset -f
import os
import shutil

# Path to the 'datasets' folder
dataset_folder = "datasets"

# List of folders to keep (Modify this if needed)
folders_to_keep = []  # Example: ["images", "labels"]

# Check if the dataset folder exists
if not os.path.exists(dataset_folder):
    print(f"Error: The folder '{dataset_folder}' does not exist.")
else:
    # Iterate through all items in the dataset folder
    for item in os.listdir(dataset_folder):
        item_path = os.path.join(dataset_folder, item)
        
        # Check if the item is not in the keep list
        if item not in folders_to_keep:
            # Remove the folder or file
            if os.path.isdir(item_path):
                shutil.rmtree(item_path)  # Remove directories
                print(f"Removed folder: {item_path}")
            else:
                os.remove(item_path)  # Remove files
                print(f"Removed file: {item_path}")

print("Cleanup completed.")




The following code downloads the COCO dataset from github.

In [ ]:
from ultralytics.utils.downloads import download
from pathlib import Path

# Define the dataset folder path
dataset_dir = Path().parent / "datasets"  # Set dataset path relative to the script

# Ensure the dataset folder exists
dataset_dir.mkdir(parents=True, exist_ok=True)

# Download labels
segments = True  # segment or box labels
url = 'https://github.com/ultralytics/assets/releases/download/v0.0.0/'
urls = [url + ('coco2017labels-segments.zip' if segments else 'coco2017labels.zip')]  # labels
download(urls, dir=dataset_dir / 'labels')

# Download images
urls = [
    'http://images.cocodataset.org/zips/train2017.zip',  # 19G, 118k images
    'http://images.cocodataset.org/zips/val2017.zip'  # 1G, 5k images
    # ,'http://images.cocodataset.org/zips/test2017.zip'  # 7G, 41k images (optional)
]
download(urls, dir=dataset_dir / 'images', threads=3)

print(f"Labels saved to {dataset_dir / 'labels'}")
print(f"Images saved to {dataset_dir / 'images'}")


# Image reordering
Copies all the images from ~/datasets/coco into the /dataset folder

In [ ]:
%reset -f
import os
import shutil
from tqdm import tqdm

# Define source directories
coco_train_images = os.path.expanduser("datasets/images/train2017")
coco_val_images = os.path.expanduser("datasets/images/val2017")
coco_train_labels = os.path.expanduser("datasets/labels/coco/labels/train2017")
coco_val_labels = os.path.expanduser("datasets/labels/coco/labels/val2017")

# Define target directories
dataset_images = "datasets/images"
dataset_labels = "datasets/labels"

# Ensure target directories exist
os.makedirs(dataset_images, exist_ok=True)
os.makedirs(dataset_labels, exist_ok=True)

# Helper function to copy files
def copy_files(source_dir, target_dir, file_extension, phase_name):
    """
    Copy files with a specific extension from source_dir to target_dir.

    Args:
        source_dir (str): Source directory path.
        target_dir (str): Target directory path.
        file_extension (str): File extension to filter.
        phase_name (str): Name of the phase (e.g., 'train', 'val') for progress.
    """
    files = [f for f in os.listdir(source_dir) if f.endswith(file_extension)]
    
    with tqdm(total=len(files), desc=f"Copying {phase_name} files", unit="file") as pbar:
        for file_name in files:
            src_path = os.path.relpath(os.path.join(source_dir, file_name))
            dst_path = os.path.relpath(os.path.join(target_dir, file_name))
            shutil.move(src_path, dst_path)
            pbar.update(1)

# Copy files
copy_files(coco_train_images, dataset_images, ".jpg", "Training Images")
copy_files(coco_val_images, dataset_images, ".jpg", "Validation Images")
copy_files(coco_train_labels, dataset_labels, ".txt", "Training Labels")
copy_files(coco_val_labels, dataset_labels, ".txt", "Validation Labels")

print("Files successfully copied to dataset/images and dataset/labels with relative paths.")




Clear unused files that have been downloaded from the COCO dataset

In [ ]:
import os

# Directories to clean
directories = [
    "datasets/images",
    "datasets/labels"
]

def clean_directory(directory, extensions=[".jpg", ".txt"]):
    """
    Removes all files and folders from a directory except those with the specified extensions.
    
    Args:
        directory (str): The directory to clean.
        extensions (list): The file extensions to keep (default: [".jpg", ".txt"]).
    """
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)
        # Check if the item is a file and doesn't have a desired extension
        if os.path.isfile(item_path) and not any(item.lower().endswith(ext) for ext in extensions):
            os.remove(item_path)  # Remove the file
        # If the item is a folder, remove it
        elif os.path.isdir(item_path):
            for root, dirs, files in os.walk(item_path, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(item_path)  # Remove the directory itself

# Clean each directory
for directory in directories:
    if os.path.exists(directory):
        clean_directory(directory)
        print(f"Cleaned directory: {directory}")
    else:
        print(f"Directory does not exist: {directory}")

print("Cleaning complete!")


### Reformat dataset to include only vehicles.
The original dataset contains the labels of:

    ["aeroplane", "bicyclebike", "bird", "boat", "bottle", "bus",
    "car", "cat", "chair", "cow", "diningtable", "dog", "horse",
    "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

A new filtered label directory has been created so that only remain the vehicles:

    ["car", "bus", "motorbike", "bicyclebike"]

These new labels are stored in dataset/labels_filtered

In [ ]:
# Directory containing YOLO label .txt files
label_dir = "datasets/labels"  # Replace with your label directory path

# Allowed Class IDs for vehicle-related objects
ALLOWED_CLASSES = {0, 1, 2, 3, 4, 5, 6, 7}  # person, bicycle, car, motorcycle, airplane, bus, train, truck

# Directory to save filtered labels
output_dir = "datasets/labels_filtered"
os.makedirs(output_dir, exist_ok=True)

def filter_labels(label_file):
    """
    Reads a YOLO label file, filters out unwanted classes,
    and writes the remaining labels to a new file.
    """
    input_path = os.path.join(label_dir, label_file)
    output_path = os.path.join(output_dir, label_file)

    with open(input_path, "r") as infile, open(output_path, "w") as outfile:
        for line in infile:
            parts = line.split()
            class_id = int(parts[0])  # Extract class ID
            if class_id in ALLOWED_CLASSES:
                # Write the line if class ID is allowed
                outfile.write(line)

# List all .txt files in the label directory
label_files = [f for f in os.listdir(label_dir) if f.endswith(".txt")]

# Process all .txt files with a progress bar
with tqdm(total=len(label_files), desc="Filtering Labels", unit="file") as pbar:
    for file_name in label_files:
        filter_labels(file_name)
        pbar.update(1)

print(f"Filtered labels saved in: {output_dir}")

### Remove empty labels

In [ ]:
# Define the path to the labels folder
labels_folder = "datasets/labels_filtered"

# List all .txt files in the labels folder
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Initialize a counter for removed files
removed_count = 0

# Check each label file and remove it if it's empty
for label_file in label_files:
    label_path = os.path.join(labels_folder, label_file)
    if os.path.getsize(label_path) == 0:  # Check if the file size is 0 bytes
        os.remove(label_path)  # Remove the empty file
        removed_count += 1
        # print(f"Removed empty label: {label_file}")

# Output the result
print(f"Total empty labels removed: {removed_count}")



Counting images and filtered_labels

In [ ]:
print(os.getcwd())
# Define the paths to the images and labels folders
images_folder = "datasets/images"
labels_folder = "datasets/labels_filtered"

# List all files in the images and labels folders
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Count the total number of images and labels
num_images = len(image_files)
num_labels = len(label_files)

# Check for matching files (base filenames without extensions)
image_basenames = {os.path.splitext(f)[0] for f in image_files}
label_basenames = {os.path.splitext(f)[0] for f in label_files}

# Count matched and unmatched files
matched_files = image_basenames & label_basenames
unmatched_images = image_basenames - label_basenames
unmatched_labels = label_basenames - image_basenames

print(f"Total images: {num_images}")
print(f"Total labels: {num_labels}")
print(f"Matched files: {len(matched_files)}")
print(f"Unmatched images: {len(unmatched_images)}")
print(f"Unmatched labels: {len(unmatched_labels)}")

# Optionally print the unmatched files
if unmatched_images:
    print("Unmatched images (no corresponding label):")
    for img in unmatched_images:
        print(f"  {img}")

if unmatched_labels:
    print("Unmatched labels (no corresponding image):")
    for lbl in unmatched_labels:
        print(f"  {lbl}")


Create the folder of images_filtered with a reduced number of unlabellel images.
The ratio of labelled images and unlabelled images has been set to 50/50. 

In [ ]:
import random

# Paths
images_folder = "datasets/images"
labels_folder = "datasets/labels_filtered"
output_folder = "datasets/images_filtered"

# Ratio of labeled and unlabeled images
r_label = 50
r_unlabel = 100 - r_label

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Get all image files and corresponding label files
image_files = [f for f in os.listdir(images_folder) if f.endswith('.jpg')]
label_files = [f for f in os.listdir(labels_folder) if f.endswith('.txt')]

# Get the base filenames (without extensions) for labels
label_basenames = {os.path.splitext(label)[0] for label in label_files}

# Separate labeled and unlabeled images
labeled_images = [img for img in image_files if os.path.splitext(img)[0] in label_basenames]
unlabeled_images = [img for img in image_files if os.path.splitext(img)[0] not in label_basenames]

# Check counts
num_labeled = len(labeled_images)
num_unlabeled_to_select = min(int(num_labeled * r_unlabel / r_label), len(unlabeled_images))

# Randomly select the required number of unlabeled images
selected_unlabeled_images = random.sample(unlabeled_images, num_unlabeled_to_select)

# Combine labeled and selected unlabeled images
images_to_copy = labeled_images + selected_unlabeled_images

# Copy labeled and selected unlabeled images to the output folder with a progress bar
with tqdm(total=len(images_to_copy), desc="Copying Images", unit="file") as pbar:
    for img in images_to_copy:
        src_path = os.path.join(images_folder, img)
        dst_path = os.path.join(output_folder, img)
        shutil.copy(src_path, dst_path)
        pbar.update(1)

# Output results
print(f"Total labeled images: {num_labeled}")
print(f"Total unlabeled images selected: {len(selected_unlabeled_images)}")
print(f"Total images in 'images_filtered': {len(os.listdir(output_folder))}")



# Create Train, Validation and Test image sets
From the image and labels ("dataset/images", "dataset/labels_filtered")
Create the test, validation and test sets.


- Training is stored in ("dataset/train/images", "dataset/train/labels")
- Validation is stored in ("dataset/valid/images", "dataset/valid/labels")
- Test is stored in ("dataset/test/images", "dataset/test/labels")


The following code takes the unfiltered images and create the corresponding training, validation and test set in the following folders:
- dataset/train
- dataset/valid
- dataset/test

In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# Split data into 20% train, 5% validation, and 5% test
train_perc = 0.8
valid_perc = 0.1
test_perc = 0.1

# Define folder paths
images_folder = "datasets/images"  # Folder containing filtered images
labels_folder = "datasets/labels"  # Folder containing filtered labels

train_images_folder = "datasets/train/images"
train_labels_folder = "datasets/train/labels"

valid_images_folder = "datasets/valid/images"
valid_labels_folder = "datasets/valid/labels"

test_images_folder = "datasets/test/images"
test_labels_folder = "datasets/test/labels"

# Create output directories
for folder in [train_images_folder, train_labels_folder,
               valid_images_folder, valid_labels_folder,
               test_images_folder, test_labels_folder]:
    if os.path.exists(folder):
        # Remove the folder and its contents
        shutil.rmtree(folder, ignore_errors=True)
    
    os.makedirs(folder, exist_ok=True)

# Get a list of all images
image_files = sorted(os.listdir(images_folder))

# Create a list of images with and without labels
data = []
for image_file in image_files:
    label_file = os.path.splitext(image_file)[0] + ".txt"
    if os.path.exists(os.path.join(labels_folder, label_file)):
        data.append((image_file, label_file))  # Image has a corresponding label
    else:
        data.append((image_file, None))  # Image has no label (no objects detected)

# Shuffle the data
random.shuffle(data)

# Calculate splits
total_data = len(data)
train_split = int(train_perc * total_data)
valid_split = train_split + int(valid_perc * total_data)
test_split = valid_split + int(test_perc * total_data)

# Allocate data
train_data = data[:train_split]
valid_data = data[train_split:valid_split]
test_data = data[valid_split:test_split]

# Function to copy images and labels with a progress bar
def copy_files(data, dest_images_folder, dest_labels_folder, phase_name):
    with tqdm(total=len(data), desc=f"Copying {phase_name}") as pbar:
        for image_file, label_file in data:
            # Copy the image file
            shutil.copy(os.path.join(images_folder, image_file), os.path.join(dest_images_folder, image_file))
            # Copy the label file if it exists
            if label_file:
                shutil.copy(os.path.join(labels_folder, label_file), os.path.join(dest_labels_folder, label_file))
            # Update progress bar
            pbar.update(1)

# Copy data to respective folders
copy_files(train_data, train_images_folder, train_labels_folder, "Training Data")
copy_files(valid_data, valid_images_folder, valid_labels_folder, "Validation Data")
copy_files(test_data, test_images_folder, test_labels_folder, "Testing Data")

print("Dataset split complete!")
print(f"Training data: {len(train_data)} images")
print(f"Validation data: {len(valid_data)} images")
print(f"Testing data: {len(test_data)} images")


The following code takes the filtered images and create the corresponding training, validation and test set in the following folders:
- dataset/train_filtered
- dataset/valid_filtered
- dataset/test_filtered


In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# Split data into 20% train, 5% validation, and 5% test
train_perc = 0.8
valid_perc = 0.1
test_perc = 0.1

# Define folder paths
images_folder = "datasets/images_filtered"  # Folder containing filtered images
labels_folder = "datasets/labels_filtered"  # Folder containing filtered labels

train_images_folder = "datasets/train_filtered/images"
train_labels_folder = "datasets/train_filtered/labels"

valid_images_folder = "datasets/valid_filtered/images"
valid_labels_folder = "datasets/valid_filtered/labels"

test_images_folder = "datasets/test_filtered/images"
test_labels_folder = "datasets/test_filtered/labels"

# Create output directories
for folder in [train_images_folder, train_labels_folder,
               valid_images_folder, valid_labels_folder,
               test_images_folder, test_labels_folder]:
    if os.path.exists(folder):
        # Remove the folder and its contents
        shutil.rmtree(folder, ignore_errors=True)
    
    os.makedirs(folder, exist_ok=True)

# Get a list of all images
image_files = sorted(os.listdir(images_folder))

# Create a list of images with and without labels
data = []
for image_file in image_files:
    label_file = os.path.splitext(image_file)[0] + ".txt"
    if os.path.exists(os.path.join(labels_folder, label_file)):
        data.append((image_file, label_file))  # Image has a corresponding label
    else:
        data.append((image_file, None))  # Image has no label (no objects detected)

# Shuffle the data
random.shuffle(data)

# Calculate splits
total_data = len(data)
train_split = int(train_perc * total_data)
valid_split = train_split + int(valid_perc * total_data)
test_split = valid_split + int(test_perc * total_data)

# Allocate data
train_data = data[:train_split]
valid_data = data[train_split:valid_split]
test_data = data[valid_split:test_split]

# Function to copy images and labels with a progress bar
def copy_files(data, dest_images_folder, dest_labels_folder, phase_name):
    with tqdm(total=len(data), desc=f"Copying {phase_name}") as pbar:
        for image_file, label_file in data:
            # Copy the image file
            shutil.copy(os.path.join(images_folder, image_file), os.path.join(dest_images_folder, image_file))
            # Copy the label file if it exists
            if label_file:
                shutil.copy(os.path.join(labels_folder, label_file), os.path.join(dest_labels_folder, label_file))
            # Update progress bar
            pbar.update(1)

# Copy data to respective folders
copy_files(train_data, train_images_folder, train_labels_folder, "Training Data")
copy_files(valid_data, valid_images_folder, valid_labels_folder, "Validation Data")
copy_files(test_data, test_images_folder, test_labels_folder, "Testing Data")

print("Dataset split complete!")
print(f"Training data: {len(train_data)} images")
print(f"Validation data: {len(valid_data)} images")
print(f"Testing data: {len(test_data)} images")


In [ ]:
import os
from collections import defaultdict
from tqdm import tqdm

# Directories to process
directories = [
    "datasets/labels",
    "datasets/train/labels",
    "datasets/valid/labels",
    "datasets/test/labels",
    
    "datasets/labels_filtered",
    "datasets/train_filtered/labels",
    "datasets/valid_filtered/labels",
    "datasets/test_filtered/labels",
]

def count_labels_in_directory(directory):
    """
    Counts the number of each class in a directory of YOLO label files with a progress bar.
    
    Args:
        directory (str): Path to the directory containing label files.
        
    Returns:
        dict: A dictionary with class IDs as keys and counts as values.
    """
    class_counts = defaultdict(int)
    
    # Get all .txt files in the directory
    label_files = [f for f in os.listdir(directory) if f.endswith(".txt")]
    
    # Process each file with a progress bar
    with tqdm(total=len(label_files), desc=f"Processing {directory}", unit="file") as pbar:
        for label_file in label_files:
            label_path = os.path.join(directory, label_file)
            with open(label_path, "r") as file:
                for line in file:
                    parts = line.split()
                    class_id = int(parts[0])  # Extract class ID
                    class_counts[class_id] += 1  # Increment the count for the class ID
            pbar.update(1)
    
    return class_counts

# Count labels for each directory
results = {}

for directory in directories:
    if os.path.exists(directory):
        results[directory] = count_labels_in_directory(directory)
    else:
        results[directory] = None  # Directory does not exist

# Display the results
print("\nClass counts per directory:")
for directory, class_counts in results.items():
    print(f"\nDirectory: {directory}")
    if class_counts is not None:
        for class_id, count in sorted(class_counts.items()):
            print(f"  Class {class_id}: {count}")
    else:
        print("  Directory does not exist or contains no labels.")



# Load YOLO Model and Begin Training!!

In [ ]:
import torch

# Check for available devices
device_name = "cuda" if torch.cuda.is_available() else "cpu"
available_devices = torch.cuda.device_count()

print(f"Device in use: {device_name}")
print(f"Available CUDA devices: {available_devices}")

In [ ]:
from ultralytics import YOLO
!yolo checks

In [ ]:
from ultralytics import YOLO

model16 = YOLO("yolo11n.yaml")

model16.train(
    data="train_filtered.yaml",
    project="./base_16",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=16,
    )

In [ ]:
from ultralytics import YOLO
model16 = YOLO("base_16/train/weights/last.pt")
model16.train(resume=True)

In [ ]:
from ultralytics import YOLO

modelX = YOLO("yolo11n.yaml")
modelX.train(
    data="train_filtered.yaml",
    project="./base_X2",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=-1,
    )

In [ ]:
from ultralytics import YOLO
model16 = YOLO("base_X/train/weights/last.pt")
model16.train(resume=True)

In [ ]:
from ultralytics import YOLO

model16 = YOLO("yolo11n.yaml")

model16.train(
    data="train_filtered.yaml",
    cfg="hyperparam_tuning2/tune/best_hyperparameters.yaml",
    project="tuned2_16",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=16,
    )

In [ ]:
from ultralytics import YOLO

model16 = YOLO("yolo11n.yaml")

model16.train(
    data="train_filtered.yaml",
    cfg="hyperparam_tuning2/tune/best_hyperparameters.yaml",
    project="tuned2_X",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=-1,
    )

In [ ]:
from ultralytics import YOLO
model16 = YOLO("tuned1_16/train/weights/last.pt")
model16.train(resume=True)

In [ ]:
from ultralytics import YOLO

model16 = YOLO("yolo11n.yaml")

model16.train(
    data="train_filtered.yaml",
    cfg="hyperparam_tuning2/tune/best_hyperparameters.yaml",
    project="tuned2_X",
    pretrained=False,  
    epochs = 200,
    patience=10, 
    batch=-1,
    )

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.yaml")

model.tune(
    data="train_filtered.yaml",
    project="./hyperparam_tuning3",
    pretrained=False,  
    epochs = 5,
    iterations=100,
    )

# Validating the YOLO models
The following code is used to validate and compare the different YOLO models that have been trained

In [ ]:
from ultralytics import settings

# View all settings
print(settings)

# Return a specific setting
value = settings["runs_dir"]
print(value)

This code tries the default and pretrained yolo11n model with the COCO dataset

In [4]:
from ultralytics import YOLO

# List of model weight paths
weights_paths = [
    "yolo11n.pt",
    "models/base_16/train/weights/best.pt",
    "models/base_40/train/weights/best.pt",
    "models/base_X/train/weights/best.pt",
    "models/tuned1_16/train/weights/best.pt",
    "models/tuned2_16/train2/weights/best.pt",
    "models/tuned2_X/train/weights/best.pt"
]

# Corresponding project names for each model
prj_paths = [
    "yolov11",
    "models/base_16",
    "models/base_40",
    "models/base_X",
    "models/tuned1_16",
    "models/tuned2_16",
    "models/tuned2_X"
]

# Path to your dataset configuration file
data_path = "train_filtered.yaml"

# Device to run the validation on (e.g., '0' for the first GPU, 'cpu' for CPU)
device = "0"

# Dictionary to store validation results
validation_results = {}

# Iterate over each model weight path and its corresponding project name
for weights_path, prj_path in zip(weights_paths, prj_paths):
    # Load the model
    model = YOLO(weights_path)
    
    # Perform validation
    results = model.val(data=data_path, project=prj_path, device=device, save_json=True, save=False)
    
    # Extract relevant metrics
    metrics = {
        'mAP50': results.box.map50,
        'mAP50-95': results.box.map,
        'mAP75': results.box.map75,
        'mAPs': results.box.maps
    }
    
    # Store metrics in the dictionary
    validation_results[weights_path] = metrics

# Print the validation results
for model_path, metrics in validation_results.items():
    print(f"\nValidation results for {model_path}:")
    print(f"  mAP50: {metrics['mAP50']:.4f}")
    print(f"  mAP50-95: {metrics['mAP50-95']:.4f}")
    print(f"  mAP75: {metrics['mAP75']:.4f}")
    print(f"  mAPs: {metrics['mAPs']}")


Ultralytics 8.3.78  Python-3.11.9 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


val: Scanning C:\Users\Usuario\Documents\TFM\ForeHelm\training\COCO\datasets\valid_filtered\labels.cache... 7641 images, 4687 backgrounds, 0 corrupt: 100%|██████████| 12328/12328 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 771/771 [01:14<00:00, 10.29it/s]


                   all      12328      36316      0.789      0.608      0.702      0.509
                person       6719      27281      0.809      0.655      0.756      0.529
               bicycle        317        724      0.728      0.445      0.529      0.326
                   car       1246       4556      0.727      0.507      0.596      0.384
            motorcycle        382       1009      0.817      0.547      0.671      0.421
              airplane        305        526      0.865      0.778       0.85      0.669
                   bus        409        679      0.819      0.713      0.801      0.653
                 train        392        522      0.885      0.769      0.852      0.673
                 truck        640       1019      0.663      0.451      0.559      0.414
Speed: 0.2ms preprocess, 2.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving yolov11\val\predictions.json...
Results saved to yolov11\val
Ultralytics 8.3.78  Python-3.11.9 torch-2.5.1+cu1

val: Scanning C:\Users\Usuario\Documents\TFM\ForeHelm\training\COCO\datasets\valid_filtered\labels.cache... 7641 images, 4687 backgrounds, 0 corrupt: 100%|██████████| 12328/12328 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 771/771 [01:11<00:00, 10.82it/s]


                   all      12328      36316      0.796        0.6      0.699      0.504
                person       6719      27281       0.83      0.657      0.768      0.542
               bicycle        317        724      0.777      0.427      0.528      0.322
                   car       1246       4556      0.735      0.518      0.611      0.395
            motorcycle        382       1009      0.811       0.55      0.673      0.418
              airplane        305        526      0.864      0.724      0.823      0.636
                   bus        409        679      0.838      0.707      0.787      0.641
                 train        392        522      0.857      0.769      0.843      0.673
                 truck        640       1019      0.656      0.447      0.556      0.406
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Saving base_16\val\predictions.json...
Results saved to base_16\val
Ultralytics 8.3.78  Python-3.11.9 torch-2.5.1+cu1

...

                   all      12328      36316      0.804      0.608       0.71       0.51
                person       6719      27281      0.841      0.656       0.77       0.54
               bicycle        317        724      0.772      0.446      0.544      0.332
                   car       1246       4556      0.753      0.519       0.62      0.397
            motorcycle        382       1009      0.806      0.548      0.669       0.42
              airplane        305        526      0.862      0.757      0.831      0.637
                   bus        409        679      0.843      0.695      0.804      0.643
                 train        392        522      0.862      0.784      0.857      0.679
                 truck        640       1019      0.691       0.46      0.582      0.428
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Saving tuned1_16\val\predictions.json...
Results saved to tuned1_16\val
Ultralytics 8.3.78  Python-3.11.9 torch-2.5.1

val: Scanning C:\Users\Usuario\Documents\TFM\ForeHelm\training\COCO\datasets\valid_filtered\labels.cache... 7641 images, 4687 backgrounds, 0 corrupt: 100%|██████████| 12328/12328 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 771/771 [01:11<00:00, 10.86it/s]


                   all      12328      36316      0.785      0.622      0.709      0.509
                person       6719      27281      0.825      0.664      0.769      0.539
               bicycle        317        724      0.745      0.456      0.544      0.332
                   car       1246       4556      0.733      0.526      0.612      0.394
            motorcycle        382       1009       0.79      0.576       0.68      0.424
              airplane        305        526      0.837       0.74      0.823      0.628
                   bus        409        679      0.836      0.722      0.804      0.646
                 train        392        522      0.863      0.794      0.859      0.678
                 truck        640       1019      0.655      0.497      0.586       0.43
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Saving tuned2_16\val2\predictions.json...
Results saved to tuned2_16\val2
Ultralytics 8.3.78  Python-3.11.9 torch-2.5

val: Scanning C:\Users\Usuario\Documents\TFM\ForeHelm\training\COCO\datasets\valid_filtered\labels.cache... 7641 images, 4687 backgrounds, 0 corrupt: 100%|██████████| 12328/12328 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 771/771 [01:07<00:00, 11.40it/s]


                   all      12328      36316      0.793       0.62      0.713      0.514
                person       6719      27281      0.821      0.673      0.772      0.544
               bicycle        317        724      0.763      0.464      0.552      0.338
                   car       1246       4556      0.727      0.535      0.619        0.4
            motorcycle        382       1009      0.792      0.576      0.691      0.437
              airplane        305        526      0.845      0.728      0.817      0.627
                   bus        409        679      0.865      0.719       0.81      0.653
                 train        392        522       0.86       0.79      0.856      0.683
                 truck        640       1019      0.672       0.48      0.588      0.432
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Saving tuned2_X\val\predictions.json...
Results saved to tuned2_X\val

Validation results for yolo11n.pt:
  mAP50: 0.

In [ ]:
# Validate the model
validation_results.box.map  # map50-95
validation_results.box.map50  # map50
validation_results.box.map75  # map75
validation_results.box.maps  # a list contains map50-95 of each category

Model Prunning

In [ ]:
import torch
import torch.nn.utils.prune as prune
from ultralytics import YOLO

def prune_model(model, amount=0.3):
    for module in model.modules():
        if isinstance(module, torch.nn.Conv2d):
            prune.l1_unstructured(module, name='weight', amount=amount)
            prune.remove(module, 'weight')
    return model

In [ ]:
# Load the YOLO model
weights = "base_40/train/weights/best.pt"
model = YOLO(weights)
pruned_model = YOLO(weights)

# Validate the original model (optional)
results_before = model.val(data="train_filtered.yaml")
print("Original model validation completed.")
torch_model = model.model

# Prune the model by updating the model's internal torch model in place
print("Pruning model...")
pruned_model.model = prune_model(torch_model, amount=0.5)
print("Model pruned.")
# Validate the pruned model using the same YOLO validation method
results_after = pruned_model.val(data="train_filtered.yaml")
print("Pruned model validation completed.")

In [ ]:
model = YOLO("base_40/train/weights/best.pt")
results = model.val(data="train_filtered.yaml")

In [ ]:

torch_model = model.model
print("Pruning model...")
pruned_torch_model = prune_model(torch_model, amount=0.2)
print("Model pruned.")
validation_results = model.val(data="train_filtered.yaml", device="0", plots=True)